In [ ]:
# %pip install --upgrade langchain langsmith langchain_openai

In [1]:
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get OpenAI keys from .env file
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_ORGANIZATION"] = os.getenv("OPENAI_ORGANIZATION")

# Initialize LangSmith
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "Agent Demo"

### Set up the Tools

In [2]:
# Import things that are needed generically for tools below
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import StructuredTool

In [3]:
class MultiplierInput(BaseModel):
    a: int = Field(description="First number")
    b: int = Field(description="Second number")


def multiply(a: int, b: int) -> int:
    return a * b


multiplier = StructuredTool.from_function(
    func=multiply,
    name="Multiplier",
    description="Multiply two numbers",
    args_schema=MultiplierInput,
    return_direct=False,
)

In [4]:
class AdderInput(BaseModel):
    a: int = Field(description="First number")
    b: int = Field(description="Second number")


def add(a: int, b: int) -> int:
    return a + b


adder = StructuredTool.from_function(
    func=add,
    name="Adder",
    description="Add two numbers",
    args_schema=AdderInput,
    return_direct=False,
)

In [5]:
tools=[multiplier, adder]

### Set up the agent

In [6]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import (
    ChatPromptTemplate,
    MessagesPlaceholder,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)

model = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0, streaming=True)

system_template = """
You are a helpful assistant.
"""

human_template = "{input}"

prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate.from_template(system_template),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        HumanMessagePromptTemplate.from_template(input_variables=["input"], template=human_template),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(model, tools, prompt)

### Let's run this agent

In [7]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
agent_executor.invoke({"input": "What is the result of adding 13 and 28, and multiplying that by 40?"})



> Entering new AgentExecutor chain...

Invoking: `Adder` with `{'a': 13, 'b': 28}`


41
Invoking: `Multiplier` with `{'a': 41, 'b': 40}`


1640The result of adding 13 and 28 is 41, and the result of multiplying 41 by 40 is 1640.

> Finished chain.


{'input': 'What is the result of adding 13 and 28, and multiplying that by 40?',
 'output': 'The result of adding 13 and 28 is 41, and the result of multiplying 41 by 40 is 1640.'}